In [ ]:
from PIL import Image
from skimage.io import imread
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.python.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from sklearn.metrics import confusion_matrix
import seaborn as sn
from datetime import datetime


In [ ]:
#We are going to organize our data by datasets so we can run multiple tests just changing the dataset name
home_path = os.getcwd()
db_path  = "refined_data"
dataset = "8_categories_3000_items"
data_path = os.path.join(home_path,db_path,dataset)
category_list=os.listdir(data_path)
category_list

In [ ]:
os.chdir(data_path)
list_of_image = glob.glob("**/*.png")
list_of_folders = glob.glob("**")
number_of_classes = str(len(list_of_folders))
print("Total number of files " + str(len(list_of_image)) )#Total data
print("Total Number of classes "+ str(len(list_of_folders))) #Total Number of classes|

In [ ]:
#INDEX for Class number and Serial Number
Class_Dict = {}
for i in range(len(list_of_folders)):
    Class_Dict[i] = os.path.basename(list_of_folders[i])

Class_Dict

In [ ]:
# Converts the class type into integers according to their order in the folder
# and appends to the list Y_list
Y_list = []
for i, data in enumerate(list_of_folders,0):
    list_of_images_in_folder = glob.glob(data+"/*.png") #check for .png
    for j in list_of_images_in_folder:
        Y_list.append(i)

Y = np.asarray(Y_list)
Y = Y.reshape(-1,1)
Y.shape

In [ ]:
# to Count the number of files in each category(i.e. folder). 
# Compare with number of files in the folder.
np.unique(Y)
unique, counts = np.unique(Y, return_counts=True)
dict(zip(unique, counts)) 

In [ ]:
image_list = []
for i,each in enumerate(list_of_image,1):
    im = imread(each, as_gray = True) #Convert images to gray and read as an array 
    image_list.append(im) #add to this list

In [ ]:
X = np.asarray(image_list) #Convert the list into array
X.shape

In [ ]:
X_ref = X #To keep the dimensions
X = X.reshape(-1, X_ref.shape[1],X_ref.shape[2], 1) #Convert to format usable by our model
X.shape

In [ ]:
#Visualise an image 
plt.imshow(X[0].reshape(X_ref.shape[1],X_ref.shape[2]), cmap = plt.cm.binary)
plt.show()

In [ ]:
#Visualisation for classes
plt.figure(figsize=(20,20));
n = 2 #Class number {0: 'nut', 1: 'locatingpin', 2: 'bolt', 3: 'washer'}
num = 100 #num of data to visualize from the cluster
for i in range(1,num): 
    plt.subplot(10, 10, i); #(Number of rows, Number of column per row, item number)
    plt.imshow(X[i+(3000*n)].reshape(X.shape[1], X.shape[2]), cmap = plt.cm.binary);
    
plt.show()

In [ ]:
#Training split
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.90,test_size=0.1, random_state=1) 

In [ ]:
print("Shape of training data " + str(X_train.shape) )
print("Shape of testing data "+ str(X_test.shape) )
print("Shape of training label "+ str(y_train.shape) )
print("Shape of testing label "+ str(y_test.shape) )

In [ ]:
Y_train_one_hot = keras.utils.to_categorical(y_train)
Y_train_one_hot.shape #One hot encoding of Y

# Sequential

In [ ]:
#Go back to main directory and checks if model, logs and plots directories exist
os.chdir(home_path)
logs_path = os.path.join(home_path, "logs")
models_path = os.path.join(home_path, "models")
plots_path = os.path.join(home_path, "plots")
if os.path.exists(logs_path) is False:
    os.mkdir(logs_path)
if os.path.exists(models_path) is False:
    os.mkdir(models_path)
if os.path.exists(plots_path) is False:
    os.mkdir(plots_path)


In [ ]:
#Creation of a CNN . Sequential Model
name = "Sequential_Model_"+datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+"_dataset_"+dataset
model = Sequential(name=name)
model.add(Conv2D(64, (3,3), input_shape=(224, 224, 1))) #input_shape matches our input image
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dense(number_of_classes)) #data of classes
model.add(Activation('softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
Y_test_one_hot = to_categorical(y_test)

In [ ]:
no_epochs = 20 #max number of epochs, should rarely get to this point
my_callbacks = [
    EarlyStopping(patience=3),
    ModelCheckpoint(filepath=os.path.join(models_path, 'model.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.h5')),
    CSVLogger(filename=os.path.join(logs_path, name + '.csv'))
]
history = model.fit(X_train, Y_train_one_hot, batch_size=64, 
                    epochs = no_epochs, 
                    validation_data=(X_test, Y_test_one_hot),
                    verbose=2,
                    callbacks=my_callbacks) #Actual Training of model

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
trained_epochs = len(history.history['loss'])
epochs = range(1,trained_epochs+1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(fname=os.path.join(plots_path, name + '_loss.png'))
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
trained_epochs = len(history.history['accuracy'])
epochs = range(1,trained_epochs+1)
plt.plot(epochs, loss_train, 'g', label='Training Accuracy')
plt.plot(epochs, loss_val, 'b', label='validation Accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(fname=os.path.join(plots_path, name + '_accuracy.png'))
plt.show()

In [ ]:
predictions = model.predict(X_test,batch_size=64, verbose=0)
y_classes = predictions.argmax(axis=-1)
cm = confusion_matrix(y_test, y_classes)
print(cm)

In [ ]:
#to print confusion matrix
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

# first generate with specified labels
cm = confusion_matrix(y_test, y_classes)
# then print it in a pretty way
print_cm(cm, category_list)

In [ ]:
array = cm
df_cm = pd.DataFrame(array, index = [i for i in category_list],
                  columns = [i for i in category_list])
plt.figure(figsize = (15,10))
sn.heatmap(df_cm, annot=True)
plt.savefig(fname=os.path.join(plots_path, name + '_confusion_matrix.png'))